<table>
    <tr>
      <td>Introducción a
      </td>
      <td>
      <img src="https://media.licdn.com/dms/image/D5612AQF7GSp3l4pztQ/article-cover_image-shrink_720_1280/0/1686548640655?e=1715817600&v=beta&t=WQzv1EMkEEwZ0QZ0PF1anRKIHCl5BBH_YPZHdDQsWPM"  width=150/>
      </td>
     </tr>
</table>
Rafa Caballero


## K-means (clustering the filas)

Ejemplo de partida

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv("https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/bus.csv")

feat_cols= ["I1","I2","I3","I4","I5","I6","I7","I8"]
df

### En busca de la K

¿Cuál es el número óptimo de grupos?

Primero lo hacemos a mano

In [ ]:
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

suma_de_cuadrados = []
K = range(1,15)
for k in K:
    kmeans = KMeans(n_clusters=k,n_init='auto')
    kmeans.fit_transform(df[feat_cols])
    suma_de_cuadrados.append(kmeans.inertia_)

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(K, suma_de_cuadrados,color="blue")
ax.set_xlabel('Valores de k')
ax.set_ylabel('Suma de distancias al centroide')
ax.grid(b=True, which='major', color='gray', linestyle='-')
plt.title('Método del codo')
plt.show()

Ahora con una librería

In [ ]:
!pip install yellowbrick --upgrade

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn import preprocessing
from yellowbrick.cluster import KElbowVisualizer


df2 =  df[feat_cols]  # df2 es df restringido a las variables de interés


# Modelo y visualizador
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12))

visualizer.fit(df2)      # Fit the data to the visualizer
visualizer.show()

Utilizamos k=3, 3 grupos

In [ ]:
k=3
model = KMeans(init='k-means++', n_clusters=k, n_init='auto')
model.fit_transform(df[feat_cols])

df2 = df.copy()
df2["C"] = model.labels_
df2["Hora"] = df["Hora"]

df2[:40]

**Ejercicio** Queremos saber cúantos hay de cada cluster

In [ ]:
for c in range(k):
    print(c,(df2.C==c).sum())

In [ ]:
def cluster_df(df):
    max = df["Hora"].max()
    min = df["Hora"].min()
    ncluster = len(df['C'].unique())
    datos = []
    horas = []
    for i in range(min,max+1):
        l = []

        for j in range(ncluster):
            n = len(df.loc[(df['C'] == j) &(df['Hora']==i) ])
            l.append(n)
        datos.append(l)
        horas.append(i)

    import pandas as pd
    df_c = pd.DataFrame(datos,columns=["C"+str(i) for i in range(1,ncluster+1)],index=horas)
    return df_c
df_c = cluster_df(df2)
df_c


Centros

In [ ]:
model.cluster_centers_

### Representación gráfica

In [ ]:
import seaborn as sns
from sklearn.manifold  import TSNE
def tsne(df):
    df_subset = df.copy()
    data_subset = df_subset[feat_cols].values
    tsne = TSNE(n_components=2, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(data_subset)
    df_subset['tsne-2d-one'] = tsne_results[:,0]
    df_subset['tsne-2d-two'] = tsne_results[:,1]
    plt.figure(figsize=(16,10))
    sns.scatterplot(
        x="tsne-2d-one", y="tsne-2d-two",
        hue="Hora",
        style = "C",
        palette=sns.color_palette("hls", totalHoras),
        sizes=(400, 4000), alpha=.9,
        data=df_subset,
        legend="full"
    )

tsne(df2)